In [1]:
!pip install -q langchain pypdf sentence-transformers chromadb openai

# 1. Multi-Query Retriever

In [2]:
# Build a sample vectorDB
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Load blog post
loader = WebBaseLoader("https://n.news.naver.com/mnews/article/003/0012317114?sid=105")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=splits, embedding=ko_embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

question = "삼성전자 갤럭시 S24는 어떨 예정이야?"

llm = ChatOpenAI(temperature=0, openai_api_key = "api_key")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [5]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 삼성 갤럭시 S24 출시 일정은 어떻게 되나요?', '2. 삼성 갤럭시 S24의 기대되는 특징은 무엇인가요?', '3. 삼성 갤럭시 S24의 스펙 및 가격에 대한 정보를 알고 싶어요.']


7

In [6]:
unique_docs

[Document(page_content="[서울=뉴시스] 삼성전자가 17일 오전 10시(현지시간, 한국 시간 18일 오전 3시) 미국 캘리포니아주 산호세(새너제이)에서 '삼성 갤럭시 언팩 2024'를 열고 갤럭시 S24를 공개한다. 사진은 포르투갈에서 유출된 갤럭시 S24 시리즈 포스터 추정 이미지 (사진=theonecid 엑스 캡처)  *재판매 및 DB 금지[서울=뉴시스]윤정민 기자 = 인공지능(AI) 서비스가 대거 탑재될 삼성전자 플래그십 스마트폰 '갤럭시 S24'가 18일 베일을 벗는다. 갤럭시 S23이 전작 대비 카메라, 디자인 등 대폭 개선됐다면, 이번 신작은", metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"}),
 Document(page_content="[서울=뉴시스] 삼성전자가 17일 오전 10시(현지시간, 한국 시간 18일 오전 3시) 미국 캘리포니아주 산호세(새너제이)에서 열 '삼성 갤럭시 언팩 2024'의 주제는 '모바일 AI의 새로운 시대 개막'이다. 앞서 삼성전자가 AI를 스마트폰 차기작 특징으로 예고했던 만큼 어떤 AI 기능이 실릴지 관심이 쏠린다.삼성전자가 공식적으로 밝힌 AI 서비스는 실시간 통화 통역이다. 이미 SK텔레콤 '에이닷' 등 통화 통역을 지원하는 앱이 있다. 하지만 자체 AI가 탑재될 갤럭시 S24는 별도 앱을 설치하지 않아도 통역 통화를 이용할 수", metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"}),
 Document(page_content="[서울=뉴시스] 8일 업계에 따르면 삼성전자는 

# 2. 기본 Parent-document Retriever

In [7]:
from langchain.retrievers import ParentDocumentRetriever

In [8]:
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [9]:
loaders = [
    PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/LLM 실습/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

In [10]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

In [11]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=ko_embedding
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [12]:
retriever.add_documents(docs, ids=None)

In [13]:
sub_docs = vectorstore.similarity_search("인공지능 예산")

In [14]:
print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

글 길이: 423


(40.0) (40.2) (48.9) (32.8) (49.8)   
하드웨어30.9 43.1 60.2 89.2 118.3 176.5 41.7%
　 (39.5) (39.7) (48.2) (32.5) (49.2)   
     자료: Global Artificial Intelligence(AI) Market, BCC Research (2022)
￮인공지능 기술을 도입하는 산업이 늘어나고 있는 점, 인공지능 분야에 진출하는 스타트업의 증가에 
따라 산업의 기술경쟁력이 높아지는 점 등은 시장성장에  촉진요인으로 작용할 전망임
￮국내 인공지능 기술에 대한 완성도가 높지 않아 국내기술의 도입처가 제한적인 점, 인공지능이 
인간의 고용 영역을 침범할 수 있고 기술의 불안정성으로 인한 사고 발생의 우려가 존재하는 점 등은 
시장성장에 저해요인 으로 작용할 전망임


In [15]:
retrieved_docs = retriever.get_relevant_documents("인공지능 예산")

In [16]:
print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 1360


| 10 | CIS이슈리포트 2022-2 호 ▶인공지능 산업의 value chain 은 ‘AI 플랫폼 공급업체 → AI 어플리케이션 개발 → AI 응용솔루션  
개발 → 이용자 ’로 구성되며 , 동 산업은 ①성장기 산업, ②대체재로부터의 위협이 낮은 산업, ③기
술집약적 산업 등의 특징을 가짐
￮알고리즘 , 하드웨어 기술개발과 응용솔루션 서비스 상용화가 활발히 진행 중인 성장기 산업이며 , 
수요 기업의 요구사항에 따라 운영플랫폼을 선택할 수 있는 구매자의 교섭력이 높은 산업임
￮직접적인 대체 기술이 없어 대체재로부터 위협이 낮은 편이며 , 알고리즘의 동작원리를 이해하고 맞
춤형 서비스를 지원하기 위한 솔루션 개발 능력이 뒷받침 되어야 하는 기술집약적 산업임
▶시장조사전문기관 BCC research 에 따르면 세계 인공지능 시장규모는 2020년 398.4 억 달러에서  
연평균 41.0% 성장하여 2025 년에는 2,223.7 억 달러의 시장을 형성할 것으로 전망됨
￮세부 솔루션 분문별로는 2020 년 기준 소프트웨어 부문의 점유율이 전체시장의 78.3% 를 차지할 
정도로 압도적으로 높음
[세계 인공지능 시장규모 ]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분 2020 2021 2022 2023 2024 2025CAGR
(2020-2025)
인공지능398.4 553.3 769.7 1,134.3 1,498.9 2,223.7 41.0%
　 (38.9) (39.1) (47.4) (32.1) (48.4)   
소프트웨어311.8 432.3 600.3 882.4 1,164.6 1,723.5 40.8%
　 (38.6) (38.8) (47.0) (32.0) (48.0)   
서비스55.7 78.0 109.3 162.6 216.0 323.7 42.2%
　 (40.0) (40.2) (48.9) (32.8) (49.8)   
하드웨어30.9 43.

# 3. 본문의 Full_chunk가 너무 길때

In [17]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=800)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=ko_embedding
)
# The storage layer for the parent documents
store = InMemoryStore()

In [18]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [19]:
retriever.add_documents(docs)

In [20]:
len(list(store.yield_keys()))

69

In [21]:
sub_docs = vectorstore.similarity_search("인공지능 예산")

In [22]:
print(sub_docs[0].page_content)

[세계 인공지능 시장규모 ]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분 2020 2021 2022 2023 2024 2025CAGR
(2020-2025)
인공지능398.4 553.3 769.7 1,134.3 1,498.9 2,223.7 41.0%


In [23]:
len(sub_docs[0].page_content)

198

In [24]:
retrieved_docs = retriever.get_relevant_documents("인공지능 예산")

In [25]:
print(retrieved_docs[0].page_content)

| 10 | CIS이슈리포트 2022-2 호 ▶인공지능 산업의 value chain 은 ‘AI 플랫폼 공급업체 → AI 어플리케이션 개발 → AI 응용솔루션  
개발 → 이용자 ’로 구성되며 , 동 산업은 ①성장기 산업, ②대체재로부터의 위협이 낮은 산업, ③기
술집약적 산업 등의 특징을 가짐
￮알고리즘 , 하드웨어 기술개발과 응용솔루션 서비스 상용화가 활발히 진행 중인 성장기 산업이며 , 
수요 기업의 요구사항에 따라 운영플랫폼을 선택할 수 있는 구매자의 교섭력이 높은 산업임
￮직접적인 대체 기술이 없어 대체재로부터 위협이 낮은 편이며 , 알고리즘의 동작원리를 이해하고 맞
춤형 서비스를 지원하기 위한 솔루션 개발 능력이 뒷받침 되어야 하는 기술집약적 산업임
▶시장조사전문기관 BCC research 에 따르면 세계 인공지능 시장규모는 2020년 398.4 억 달러에서  
연평균 41.0% 성장하여 2025 년에는 2,223.7 억 달러의 시장을 형성할 것으로 전망됨
￮세부 솔루션 분문별로는 2020 년 기준 소프트웨어 부문의 점유율이 전체시장의 78.3% 를 차지할 
정도로 압도적으로 높음
[세계 인공지능 시장규모 ]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분 2020 2021 2022 2023 2024 2025CAGR
(2020-2025)
인공지능398.4 553.3 769.7 1,134.3 1,498.9 2,223.7 41.0%
　 (38.9) (39.1) (47.4) (32.1) (48.4)


In [26]:
len(retrieved_docs[0].page_content)

797

# 4. Self-querying

In [27]:
!pip install lark

In [29]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, ko_embedding)

In [30]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chat_models import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(temperature=0, openai_api_key = "api_key")
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose = True
)

In [31]:
retriever.get_relevant_documents("what are some movies rated higher than 8.5")

[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006})]

# 5. Time-weighted vector store Retriever

Scoring 방법 = *semantic_similarity + (1.0 - decay_rate) ^ hours_passed*

In [32]:
!pip install -q faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.6 MB/s eta 0:00:00


In [33]:
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import Document
from langchain_community.vectorstores import FAISS


In [34]:
# Initialize the vectorstore as empty
embedding_size = 768
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(ko_embedding, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.99, k=1
)

In [35]:
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents(
    [Document(page_content="영어는 훌륭합니다.", metadata={"last_accessed_at": yesterday})]
)
retriever.add_documents([Document(page_content="한국어는 훌륭합니다")])

['af4bd04c-b0d3-47ec-8659-4ea5f85db2e2']

In [36]:
# "Hello World" is returned first because it is most salient, and the decay rate is close to 0., meaning it's still recent enough
retriever.get_relevant_documents("영어가 좋아요")

[Document(page_content='한국어는 훌륭합니다', metadata={'last_accessed_at': datetime.datetime(2024, 5, 4, 7, 58, 5, 747670), 'created_at': datetime.datetime(2024, 5, 4, 7, 58, 5, 194929), 'buffer_idx': 1})]

# 6. Ensemble Retriever

In [37]:
!pip install -q langchain pypdf sentence-transformers chromadb langchain-openai faiss-gpu --upgrade --quiet  rank_bm25 > /dev/null

In [38]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [39]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/LLM 실습/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [42]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2



embedding = ko_embedding
faiss_vectorstore = FAISS.from_documents(texts, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [43]:
docs = ensemble_retriever.invoke("혁신정책금융과 극저신용대출모형의 차이")
for i in docs:

  print(i.metadata)
  print(":")
  print(i.page_content)
  print("-"*100)

{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 17}
:
기관 뿐만 아니라 전 금융권으로 혁신기업에 대한 금융지원을 확대할 필요가 있음 
※ 본 보고서의 내용은 작성자 개인의 의견으로서 한국신용정보원의 공식 견해와 다를 수 있습니다 .  
   본 보고서를 사용 또는 인용할 경우에는 출처를 명시하시기 바랍니다 .
----------------------------------------------------------------------------------------------------
{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf', 'page': 4}
:
현장공감 경기복지재단 05
Ⅰ. 경기도 극저신용대출심사모형 개발의 필요성대출의 경우 신용점수 하위 30%( 기존 5등급이하 ) 차주 중 중신용층에게 유리한 공급 
개선방안을 제시함 (금융위원회 , 2021)
-상대적으로 불량률과 연체율이 높은 7등급 이하 저신용자들은 부실위험이 높다는 이유
로 여전히 정부와 금융권의 금융거래 기피대상으로 분류되어 실질적인 대출 혜택이 불
가능한 상황이며 저신용 금융취약도민이 거의 유일하게 이용할 수 있는 제도권 금융상
품은 극저신용대출사업뿐임 (석희정 외, 2021)
내용 1차(2020 년 4월∽) 2차(2020 년 7월∽) 3차(2020 년 10월∽)
대출금액긴급(무심사 )(50만 원)
일반(심사)(300 만 원)무심사 (50만 원)
심사(300만 원)
불법사금융 피해자 (최대 300만 원)고금리 이용자 (300만 

In [44]:
faiss_vectorstore = FAISS.from_documents(texts, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

docs = faiss_retriever.invoke("혁신정책금융과 극저신용대출모형의 차이")
for i in docs:

  print(i.metadata)
  print(":")
  print(i.page_content)
  print("-"*100)

{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf', 'page': 4}
:
현장공감 경기복지재단 05
Ⅰ. 경기도 극저신용대출심사모형 개발의 필요성대출의 경우 신용점수 하위 30%( 기존 5등급이하 ) 차주 중 중신용층에게 유리한 공급 
개선방안을 제시함 (금융위원회 , 2021)
-상대적으로 불량률과 연체율이 높은 7등급 이하 저신용자들은 부실위험이 높다는 이유
로 여전히 정부와 금융권의 금융거래 기피대상으로 분류되어 실질적인 대출 혜택이 불
가능한 상황이며 저신용 금융취약도민이 거의 유일하게 이용할 수 있는 제도권 금융상
품은 극저신용대출사업뿐임 (석희정 외, 2021)
내용 1차(2020 년 4월∽) 2차(2020 년 7월∽) 3차(2020 년 10월∽)
대출금액긴급(무심사 )(50만 원)
일반(심사)(300 만 원)무심사 (50만 원)
심사(300만 원)
불법사금융 피해자 (최대 300만 원)고금리 이용자 (300만 원)
2030 청년(300만 원)
심사(300만 원 한도)
이자율 연 1%
----------------------------------------------------------------------------------------------------
{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf', 'page': 3}
:
포용금융의 성격
-특히 중앙정부에서 실행하는 서민금융대출제도는 주로 신용등급기준 4

In [45]:
import os
os.environ["OPENAI_API_KEY"] = 'api_key'

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)

qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",
                                 retriever = ensemble_retriever,
                                 return_source_documents = True)

query = "극저신용자 대출의 신용등급"
result = qa(query)
print(result['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


극저신용자 대출의 신용등급은 주로 7등급 이하로 분류됩니다. 이는 일반적인 10등급 기준에서 하위에 해당하는 등급으로, 금융이력이 부족하거나 불량 건이 많은 경우에 해당할 수 있습니다.


In [46]:
for i in result['source_documents']:
  print(i.metadata)
  print("-"*100)
  print(i.page_content)
  print("-"*100)

{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf', 'page': 11}
----------------------------------------------------------------------------------------------------
현장공감 경기복지재단 12
경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색-2021년 6월말 기준 신용등급 대상자 (47,307,806 명) 가운데 27.1%(12,807.275 명)이 금
융이력부족자로 나타났고 , 60대 이상(4,179,087 명), 20대(3,227,319 명), 30대(1,723,466
명), 50대(1,450,496 명) 순으로 20대 청년과 60대 이상이 절반 이상을 차지함
-더구나 대다수 금융이력부족자들은 700점대의 낮은 신용점수 (CB등급 기준 4~7등급)를 받는 
것으로 나타나 금융이력부족군 중 상당수의 저신용 금융취약계층이 분포되어 있음을 확인
-결과적으로 금융거래정보 위주의 개인신용평가모형은 금융소외계층에 대한 금융불평등 
문제를 지속적으로 양산할 수밖에 없는 한계를 지니고 있어 이들을 위한 더욱 정교하
고 공평한 신용평가모형 개발에 대한 요구 증대
 평점 전국민대상자금융이력부족자 현황
----------------------------------------------------------------------------------------------------
{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을

In [47]:
faiss_vectorstore = FAISS.from_documents(docs, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",
                                 retriever = faiss_retriever,
                                 return_source_documents = True)

query = "극저신용자 대출의 신용등급"
result = qa(query)
print(result['result'])

극저신용자 대출의 경우, 일반적으로 7등급 이하의 신용등급을 가진 사람들이 해당됩니다. 이는 기존의 10등급 기준에서 7등급 이하로 분류된 극저신용자를 대상으로 하는 것으로 보입니다.


In [48]:
for i in result['source_documents']:
  print(i.metadata)
  print("-"*100)
  print(i.page_content)
  print("-"*100)

{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf', 'page': 6}
----------------------------------------------------------------------------------------------------
현장공감 경기복지재단 07
Ⅰ. 경기도 극저신용대출심사모형 개발의 필요성◯도내 저신용 ･저소득층의 극저신용대출 수요증대에 따라 대출신청 시 금융 및 비
금융 정보수집에 기반한 변별력을 갖춘 심사체계 마련이 요구되는 상황
-경기극저신용대출은 기존 신용평가체계 (10등급 기준)를 활용하여 신용정보회사 (NICE, 
KCB) 에서 제공하는 10등급 기준 7등급 이하의 극저신용자를 모집단으로 설정, 극저신
용대출 수행기관 (사회연대은행 , 주빌리은행 , 지역자활협회 )을 통해 신청 및 심사절차를 
거쳐 최종 대상자를 선정함
-현행 극저신용대출 심사기준 (2020) 에 따르면 차주 관련 기초현황점수 (정량지표 , 55점)
와 심사점수 (정성지표 , 45점)로 구분되어 있고, 심사기준별 세부심사항목은 인구학적 
정보(성별, 연령, 거주지 , 직업, 가구원 , 주거형태 ), 경제상황 (정부지원 여부, 소득유무 ,
----------------------------------------------------------------------------------------------------
{'source': '/content/drive/MyDrive/Colab Notebooks/LLM 실습/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 ᄒ

# 7. Long Context Reorder

In [53]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "보스턴 셀틱스에 관한 문서입니다.", "보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.",
    "저는 영화 보러 가는 것을 좋아해요",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적 인 NBA 선수였습니다.",
]

# Create a retriever
retriever = Chroma.from_texts(texts, embedding=ko_embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱스에 대해 어떤 이야기를 들려주시겠어요?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='보스턴 셀틱스에 관한 문서입니다.'),
 Document(page_content='윤정민 기자(alpaca@newsis.com)\n\n\n\n\n기자 프로필\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n윤정민 기자\n\n\n\n\n\nTALK\n\n유익하고 소중한 제보를 기다려요!\n제보\n\n닫기\n\n\n\n\n구독\n\n\n구독중\n\n\n\n\n\n구독자 0\n\n\n\n\n응원수\n0\n\n\n\n\n\n\n\nOTT, 통신, AI, 과학 등 IT의 모든 것을 취재합니다. 많이 배우고 듣겠습니다.\n\n\n\n\n야구 유료 중계 오늘 시작인데…스포츠팬 78% "OTT 유료 중계, 싫다"\n\n\n"iOS에 들어갈 AI는 어디?"…애플-오픈AI, 아이폰용 AI 도입 재논의', metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"}),
 Document(page_content='언론사를 함께 구독중입니다.\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\n“여기 가면 류현진 볼까?”...한화이글스 특화 편의점 나왔다는데\n매일경제\n1일전\n\n\n\n\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n새로보기\n\n\n\n\n\n함께 볼만한 뉴스\n안내\n\n이 기사를 본 이용자들이 함께 많이 본 기사, 해당 기사와 유사한 기사, 관심 기사 등을 자동 추천합니다\n닫기', metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"

In [54]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(page_content='윤정민 기자(alpaca@newsis.com)\n\n\n\n\n기자 프로필\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n윤정민 기자\n\n\n\n\n\nTALK\n\n유익하고 소중한 제보를 기다려요!\n제보\n\n닫기\n\n\n\n\n구독\n\n\n구독중\n\n\n\n\n\n구독자 0\n\n\n\n\n응원수\n0\n\n\n\n\n\n\n\nOTT, 통신, AI, 과학 등 IT의 모든 것을 취재합니다. 많이 배우고 듣겠습니다.\n\n\n\n\n야구 유료 중계 오늘 시작인데…스포츠팬 78% "OTT 유료 중계, 싫다"\n\n\n"iOS에 들어갈 AI는 어디?"…애플-오픈AI, 아이폰용 AI 도입 재논의', metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"}),
 Document(page_content="언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\n\n\n\nNAVER\n\n뉴스\n\n\n연예\n\n\n\n\n스포츠\n\n\n\n\n날씨\n\n\n\n\n프리미엄\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n언론사별\n\n\n정치\n\n\n경제\n\n\n사회\n\n\n생활/문화\n\n\nIT/과학\n\n\n세계\n\n\n랭킹\n\n\n신문보기\n\n\n오피니언\n\n\nTV\n\n\n팩트체크\n\n\n알고리즘 안내\n\n\n정정보도 모음\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n구독", metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/001

In [55]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
import os
os.environ["OPENAI_API_KEY"] = 'api_key'

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

template = """Given this text extracts:
-----
{context}
-----
Please answer the following question:
{query}"""
prompt = PromptTemplate(
    template=template, input_variables=["context", "query"]
)
openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)

llm_chain = LLMChain(llm=openai, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context"
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [56]:
reordered_result = chain.run(input_documents=reordered_docs, query=query)
result = chain.run(input_documents=docs, query=query)

print(reordered_result)
print("-"*100)
print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


셀틱스는 제가 가장 좋아하는 팀입니다.
----------------------------------------------------------------------------------------------------
셀틱스는 제가 가장 좋아하는 팀이며, L. 코넷은 최고의 셀틱스 선수 중 한 명이라고 이야기하고 있습니다.
